In [1]:
!pip install rasterio -q
!pip install pandas_path --quiet

import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_path import path
from pathlib import Path
from PIL import Image
import rasterio

import os
import random
import numpy.ma as ma
plt.style.use("ggplot")
%matplotlib inline

from IPython.display import HTML, display

!pip install line_profiler

import cv2
import albumentations
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.metrics import MeanIoU
import keras.backend as K
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

pd.set_option('max_colwidth', 400)

     |████████████████████████████████| 19.3 MB 1.5 MB/s 
     |████████████████████████████████| 63 kB 86 kB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data path

In [3]:
img_size = 512

DATA_DIR = Path.cwd().parent / "content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train"
TRAIN_FEATURES = DATA_DIR / "train_features"
TRAIN_LABELS = DATA_DIR / "train_labels"

BANDS = ["B02", "B03", "B04", "B08"]

train_meta = pd.read_csv(DATA_DIR / "train_metadata_updated.csv", index_col=0)

In [4]:
def add_paths(df, feature_dir, label_dir=None, bands=BANDS):
    """
    Given dataframe with a column for chip_id, returns a dataframe with a column
    added indicating the path to each band's TIF image as "{band}_path", eg "B02_path".
    A column is also added to the dataframe with paths to the label TIF, if the
    path to the labels directory is provided.
    """
    for band in bands:
        df[f"{band}_path"] = ""

    for i in range(len(df)):
      for band in bands:
          df[f"{band}_path"][i] = feature_dir / df["chip_id"][i] / f"{band}.tif"
          #assert df[f"{band}_path"].path.exists().all()
      if label_dir is not None:
          df["label_path"][i] = label_dir / (df["chip_id"][i] + ".tif")
          #assert df["label_path"].path.exists().all()

    return df

In [5]:
train_meta = add_paths(train_meta, TRAIN_FEATURES, TRAIN_LABELS, bands=BANDS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [6]:
train_meta.head(5)

,chip_id,location,datetime,cloudpath,B02_path,B03_path,B04_path,B08_path,label_path,cloud_cover
0,adwp,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwp,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwp.tif,0.479294
1,adwu,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwu,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwu.tif,0.636658
2,adwz,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwz,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwz.tif,1.000000
3,adxp,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adxp,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adxp.tif,1.000000
4,aeaj,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/aeaj,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/aeaj.tif,1.000000


## Split the data

In [9]:
test_filenames = pd.read_csv(DATA_DIR / "test_split.csv")['location'].to_list()
test_filenames

['Adelaide',
 'Alto Hama',
 'Angkor Wat',
 'Australia - Central',
 'Australia - North West',
 'Bambesa',
 'Bechar',
 'Bor',
 'Cabo Verdo',
 'Chifunfu',
 'Chingola',
 'Ecuador',
 'Eritrea',
 'Gabon',
 'Ghana',
 'Harare']

In [13]:
df_train = train_meta.loc[~train_meta['location'].isin(test_filenames)]
df_test = train_meta.loc[train_meta['location'].isin(test_filenames)]

print("Test data ratio", round(100*len(df_test)/len(train_meta), 2), "%")

Test data ratio 20.66 %


## Prepare loading the data

In [ ]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)

In [ ]:
def load_label(image_index):
    image = np.zeros((512,512,1))
    with rasterio.open(train_meta['label_path'][image_index]) as img:            
        image[:,:,0] = img.read(1)
    return image

def cover_percentage(image_index):
    img = load_label(image_index)
    return np.sum(img)/(512*512)